<a href="https://colab.research.google.com/github/CandyAlisbo/Object-Measurement-using-OpenCV/blob/main/Director%2CActor%2CGenre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
movies = pd.read_csv('oscars_df.csv',encoding='latin-1')

features = ['Actors', 'Directors', 'Genre']

def clean_data(x):
  if isinstance(x, list):
    return [str.lower(i.replace(" ", ""),(",", "")) for i in x]
  else:
    if isinstance(x, str):
      return str.lower(x.replace(" ", ""))
    else:
      return ''

for feature in features:
  movies[feature] = movies[feature].apply(clean_data)

def create_soup(x):
    return ''.join(x['Directors']) + ' ' + ''.join(x['Actors']) + ' '  + ''.join(x['Genre'])

movies['Actors'] = movies['Actors'].str.replace(',',' ')
movies['Directors'] = movies['Directors'].str.replace(',',' ')
movies['Genre'] = movies['Genre'].str.replace(',',' ')

movies['soup'] = movies.apply(create_soup, axis=1)

#movies[['soup']].head(2)

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movies['soup'])

#count_matrix.shape

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['Film'])

#indices[:10]

def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:16]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies['Film'].iloc[movie_indices]

get_recommendations('The Godfather')

290       The Godfather Part II
373      The Godfather Part III
234       To Kill a Mockingbird
292            The Conversation
317              Apocalypse Now
436         Lost in Translation
568       Promising Young Woman
190           On the Waterfront
179    A Streetcar Named Desire
208                    Sayonara
580             West Side Story
557                The Irishman
297           Dog Day Afternoon
22              Five Star Final
206                12 Angry Men
Name: Film, dtype: object